<a href="https://colab.research.google.com/github/SKKU-SecLab/MiningPerturbation/blob/main/Adversarial_Perturbation_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install tsfresh
# !pip install scapy

In [ ]:
import pandas as pd
import tsfresh
import os
import json
import scapy
import numpy as np
import warnings
from timeit import default_timer as timer
from scapy.all import *

import sys
from pathlib import Path

warnings.filterwarnings("ignore") 

In [ ]:
def dataframe_from_csv(target):
    return pd.read_csv(target).rename(columns=lambda x: x.strip())

def dataframe_from_csvs(targets):
    return pd.concat([dataframe_from_csv(x) for x in targets])

In [ ]:
def run_process(a,b,x, changed_mal):
    
    df_malicious = a.copy()
    df_benign    = b.copy()
    
    df_mal_web_mine = changed_mal.copy()

    from tsfresh import extract_features, select_features
    from tsfresh.utilities.dataframe_functions import impute
    from tsfresh import extract_features
    from tsfresh.feature_selection.relevance import calculate_relevance_table

    df_malicious.reset_index(drop=True, inplace=True) #reset index
    df_malicious['id']= np.floor(df_malicious.index.array/10)
    df_benign.reset_index(drop=True, inplace=True) #reset index
    df_benign['id']= np.floor(df_benign.index.array/10)

    df_mal_web_mine.reset_index(drop=True, inplace=True) #reset index
    df_mal_web_mine['id']= np.floor(df_mal_web_mine.index.array/10)
    
    tf1=tsfresh.extract_features(df_malicious,impute_function=impute, column_kind='Is_malicious',
                                 column_id='id',column_sort="Time",column_value = "Length")
    tf1['class']= 1

    tf2=tsfresh.extract_features(df_benign,impute_function=impute, column_kind='Is_malicious',
                                 column_id='id',column_sort="Time",column_value = "Length")
    tf2['class']= 0

    tf3=tsfresh.extract_features(df_mal_web_mine,impute_function=impute, column_kind='Is_malicious',
                                 column_id='id',column_sort="Time",column_value = "Length")
    tf3['class']= 1

    tf2.columns = tf1.columns
    tf3.columns = tf1.columns

    features=pd.concat([tf1,tf2])
    features_changed_mal = pd.concat([tf3,tf2])

    features2 = features.copy()
    features_changed_mal2 = features_changed_mal.copy()

    features2.reset_index(drop=True, inplace=True)
    features_changed_mal2.reset_index(drop=True, inplace=True)

    y = pd.Series(data = features2['class'], index=features2.index)
    y_changed_mal = pd.Series(data = features_changed_mal2['class'], index=features_changed_mal2.index)

    from tsfresh.examples import load_robot_execution_failures
    from tsfresh import extract_features, select_features
    from tsfresh.feature_selection.relevance import calculate_relevance_table

    relevance_table = calculate_relevance_table(features2, y)
    relevance_table = relevance_table[relevance_table.relevant]
    relevance_table.sort_values("p_value", inplace=True)
    print("relevance_table " , relevance_table.shape)
    relevance_table_changed_mal = calculate_relevance_table(features_changed_mal2, y_changed_mal)
    relevance_table_changed_mal = relevance_table_changed_mal[relevance_table_changed_mal.relevant]
    relevance_table_changed_mal.sort_values("p_value", inplace=True)
    print("relevance_table_changed_mal " , relevance_table_changed_mal.shape)

    best_features = pd.DataFrame(relevance_table[relevance_table['p_value'] <= 0.05])
    # print("best_features " , best_features.shape)
    # print("best_features features" , best_features.feature)

    mal_pktPlus_features = pd.DataFrame(relevance_table_changed_mal)

    if (len(mal_pktPlus_features) >= len(best_features)):
      mal_pktPlus_features = mal_pktPlus_features[:len(best_features)]
    else:
      best_features = best_features[:len(mal_pktPlus_features)]
      # mal_pktPlus_features = pd.concat([mal_pktPlus_features, best_features[~best_features.feature.isin(mal_pktPlus_features.feature)]])
      # mal_pktPlus_features = mal_pktPlus_features.iloc[:len(best_features)]
    df_ML = pd.DataFrame()
    df_ML_mal = pd.DataFrame()

    for pkt in best_features:
        df_ML[best_features.feature] = features[best_features.feature]
        # df_ML_mal[best_features.feature] = features_changed_mal2[mal_pktPlus_features.feature]
      
    for pkt in mal_pktPlus_features:
        df_ML_mal[mal_pktPlus_features.feature] = features_changed_mal2[mal_pktPlus_features.feature]
    # mal_pktPlus_features.feature = relevance_table_changed_mal.feature

    # print("mal_pktPlus_features " , mal_pktPlus_features.shape)
    # print("mal_pktPlus_features features" , mal_pktPlus_features.feature)

    # df_ML.columns = df_ML_mal.columns    

    final = ML_Process(df_ML,x, df_ML_mal)

    return final

In [ ]:
def ML_Process(df_ML,x, df_ML_mal):
    df_results = x.copy() 
    print('let the ml starts')
  
    from sklearn import neighbors, metrics
    from sklearn.preprocessing import LabelEncoder

    #X = df_finalized[['Time', 'Length','Protocol']].values
    X = df_ML.drop('class',axis=1).to_numpy()
    X_mal = df_ML_mal.drop('class',axis=1).to_numpy()
    #y = df_finalized[['Is_malicious']]
    y = df_ML['class'].to_numpy()
    y_mal = df_ML_mal['class'].to_numpy()

    #print(X,y)
    
    # from sklearn.model_selection import train_test_split
    Le = LabelEncoder()
    for i in range(len(X[0])):
        X[:, i] = Le.fit_transform(X[:, i])
    for i in range(len(X_mal[0])):
        X_mal[:, i] = Le.fit_transform(X_mal[:, i])
    
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=8675309)
    X_train = X
    y_train = y
    X_test = X_mal
    y_test = y_mal

    from sklearn.svm import SVC
    from sklearn import model_selection
    from sklearn.utils import class_weight
    from sklearn.metrics import classification_report
    from sklearn.metrics import confusion_matrix
    import numpy as np
    import pandas as pd
    y_train = y_train.ravel()
    dfs = []
    models = [
          ('SVM', SVC()), ##### Tekiner et al., SVM performs the best among all the classifiers in terms of all metrics
            ]
    results = []
    names = []
    scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
    target_names = ['malignant', 'benign']
    for name, model in models:
        kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=None)
        cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, 
                                                    scoring=scoring)
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print(name)
        print(classification_report(y_test, y_pred, target_names=target_names))
        results.append(cv_results)
        names.append(name)
        this_df = pd.DataFrame(cv_results)
        this_df['model'] = name
        dfs.append(this_df)
        df_resulta = df_results.append(dfs)
        final = pd.concat(dfs, ignore_index=True)
        
    return(final)

In [ ]:
def insertion(df, interval):
  df.reset_index(drop=True)
  timeline = df['Time'].astype(int).unique()
  insert_packet = pd.DataFrame([['Time', 'sip', 'dip', 'proto', 74, 'hw_src', 'hw_dst', 1]], columns = df_malicious.columns)

  for i in range(len(timeline)):
    if ((timeline[i] % interval) == 0):
      insert_packet['Time'] = timeline[i]
      df = pd.concat([df, insert_packet], ignore_index = True)

In [ ]:
def fragment_man(df, split_val):
  split = pd.DataFrame(columns = df.columns)
  split_value = split_val

  for i in range(len(df)):
    if (df['Length'].iloc[i] > split_value):
      df['Length'].iloc[i] = split_value
      for j in range((df['Length'].iloc[i].astype(int) // split_value)):
        df = df.append(df.iloc[i], ignore_index = True)

##### PERTURBATION (Single Method)

In [ ]:
### DUMMY PACKET
df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/single_method/dummy_packet.csv', index_col = [0])

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()
print(end - start)

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [01:00<00:00,  3.04s/it]


relevance_table  (255, 4)
relevance_table_changed_mal  (251, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.50      1.00      0.67      6260
      benign       0.00      0.00      0.00      6186

    accuracy                           0.50     12446
   macro avg       0.25      0.50      0.33     12446
weighted avg       0.25      0.50      0.34     12446

285.06597892900004


In [ ]:
### PADDING
df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/single_method/padding.csv', index_col = [0])

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()
print(end - start)

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [00:54<00:00,  2.71s/it]


relevance_table  (255, 4)
relevance_table_changed_mal  (258, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.51      0.99      0.67      6260
      benign       0.64      0.02      0.04      6055

    accuracy                           0.51     12315
   macro avg       0.58      0.51      0.36     12315
weighted avg       0.58      0.51      0.36     12315

258.4959546320001


In [ ]:
### SPLITTING
df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/single_method/splitting.csv', index_col = [0])

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()
print(end - start)

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [01:02<00:00,  3.14s/it]


relevance_table  (255, 4)
relevance_table_changed_mal  (260, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.48      1.00      0.65      6260
      benign       0.00      0.00      0.00      6858

    accuracy                           0.48     13118
   macro avg       0.24      0.50      0.32     13118
weighted avg       0.23      0.48      0.31     13118

264.5656621569997


In [ ]:
### OBFS4 PROXY
df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/single_method/obfs4_mining.csv', index_col = [0])

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()
print(end - start)

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [00:52<00:00,  2.65s/it]


relevance_table  (255, 4)
relevance_table_changed_mal  (258, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.50      1.00      0.67      6260
      benign       0.00      0.00      0.00      6305

    accuracy                           0.50     12565
   macro avg       0.25      0.50      0.33     12565
weighted avg       0.25      0.50      0.33     12565

262.9792159140002


##### PERTURBATION (Combination)

In [ ]:
### DUMMY PACKET & PADDING
df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/combining_2_method/dummy_packet_padding.csv', index_col=[0])

df_mal_web_mine.reset_index(drop=True)

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [00:52<00:00,  2.62s/it]


relevance_table  (255, 4)
relevance_table_changed_mal  (251, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.50      1.00      0.67      6260
      benign       0.00      0.00      0.00      6186

    accuracy                           0.50     12446
   macro avg       0.25      0.50      0.33     12446
weighted avg       0.25      0.50      0.34     12446



In [ ]:
### DUMMY PACKET & SPLITTING
df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/combining_2_method/dummy_packet_splitting.csv', index_col = [0])

df_mal_web_mine.reset_index(drop=True)

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [00:55<00:00,  2.76s/it]


relevance_table  (255, 4)
relevance_table_changed_mal  (261, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.49      1.00      0.66      6260
      benign       0.00      0.00      0.00      6436

    accuracy                           0.49     12696
   macro avg       0.25      0.50      0.33     12696
weighted avg       0.24      0.49      0.33     12696



In [ ]:
### PADDING & SPLITTING
df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/combining_2_method/padding_splitting.csv', index_col = [0])

df_mal_web_mine.reset_index(drop=True)

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [01:02<00:00,  3.12s/it]


relevance_table  (255, 4)
relevance_table_changed_mal  (262, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.50      1.00      0.67      6260
      benign       0.00      0.00      0.00      6305

    accuracy                           0.50     12565
   macro avg       0.25      0.50      0.33     12565
weighted avg       0.25      0.50      0.33     12565



In [ ]:
### DUMMY PACKET & PADDING & SPLITTING
df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/combining_2_method/dummy_packet_padding_splitting.csv',index_col = [0])

df_mal_web_mine.reset_index(drop=True)

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [01:27<00:00,  4.37s/it]


relevance_table  (255, 4)
relevance_table_changed_mal  (266, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.38      1.00      0.56      6260
      benign       0.00      0.00      0.00     10012

    accuracy                           0.38     16272
   macro avg       0.19      0.50      0.28     16272
weighted avg       0.15      0.38      0.21     16272



In [ ]:
# benign_data = sorted([x for x in Path("/content/drive/MyDrive/src/datasets/IoT Cryptojacking paper/benign-2/Raspberry").glob("*.csv")])# ------------------ Check
# benign = dataframe_from_csvs(benign_data)

In [ ]:
# df_benign_add = pd.read_csv('/content/drive/MyDrive/src/datasets/Cryptojacking/RASP/benign/for_tsfresh_benign.csv')
# df_benign_add2 = pd.read_csv('/content/drive/MyDrive/src/datasets/Cryptojacking/RASP/benign/benign_webbrowsing.csv')
# df_mal_thr50_1 = pd.read_csv('/content/drive/MyDrive/src/datasets/Cryptojacking/RASP/throttle50/50_1/for_tsfresh_moneromine_50_1.csv')
# df_mal_thr50_2 = pd.read_csv('/content/drive/MyDrive/src/datasets/Cryptojacking/RASP/throttle50/50_2/for_tsfrsh_moneromine_50_2.csv')
# df_mal_thr100 = pd.read_csv('/content/drive/MyDrive/src/datasets/Cryptojacking/RASP/throttle100/for_tsfresh_moneromine_100.csv')
# df_mal_obfs4 = pd.read_csv('/content/drive/MyDrive/src/datasets/Cryptojacking/obfs4_mining.csv')

# print("df_benign_add -->> {}".format(len(df_benign_add)))
# print("df_benign_add2 -->> {}".format(len(df_benign_add2)))
# print("df_mal_thr50_1 -->> {}".format(len(df_mal_thr50_1)))
# print("df_mal_thr50_2 -->> {}".format(len(df_mal_thr50_2)))
# print("df_mal_thr100 -->> {}".format(len(df_mal_thr100)))
# print("df_mal_obfs4 -->> {}".format(len(df_mal_obfs4)))

# df_benign_add.columns = ['Time', 'Length']
# df_benign_add2.columns = ['Time', 'Length']
# df_mal_thr50_1.columns = ['Time', 'Length']
# df_mal_thr50_2.columns = ['Time', 'Length']
# df_mal_thr100.columns = ['Time', 'Length']
# df_mal_obfs4.columns = ['Time', 'Length']

# df_benign_add.insert(2, "Is_malicious", 0)
# df_benign_add2.insert(2, "Is_malicious", 0)
# df_mal_thr50_1.insert(2, "Is_malicious", 1)
# df_mal_thr50_2.insert(2, "Is_malicious", 1)
# df_mal_thr100.insert(2, "Is_malicious", 1)
# df_mal_obfs4.insert(2, "Is_malicious", 1)

# print("df_benign_add -->> {}".format(len(df_benign_add.dropna())))
# print("df_benign_add2 -->> {}".format(len(df_benign_add2.dropna())))
# print("df_mal_thr50_1 -->> {}".format(len(df_mal_thr50_1.dropna())))
# print("df_mal_thr50_2 -->> {}".format(len(df_mal_thr50_2.dropna())))
# print("df_mal_thr100 -->> {}".format(len(df_mal_thr100.dropna())))
# print("df_mal_obfs4 -->> {}".format(len(df_mal_obfs4.dropna())))

In [ ]:
### OBFS4 PROXY & DUMMY PACKET

df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic2.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic2.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/combining_2_method/dummy_packet_obfs4.csv', index_col = [0])

df_mal_web_mine.reset_index(drop=True)

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [01:03<00:00,  3.19s/it]


relevance_table  (247, 4)
relevance_table_changed_mal  (241, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.47      1.00      0.64      6701
      benign       0.00      0.00      0.00      7540

    accuracy                           0.47     14241
   macro avg       0.24      0.50      0.32     14241
weighted avg       0.22      0.47      0.30     14241



In [ ]:
### OBFS4 PROXY & PADDING

df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic2.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic2.csv', index_col = [0])

df_mal_web_mine = pd.read_csv('./attacked_data/combining_2_method/padding_obfs4.csv', index_col = [0])

df_mal_web_mine.reset_index(drop=True)

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [00:58<00:00,  2.91s/it]


relevance_table  (247, 4)
relevance_table_changed_mal  (254, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.49      1.00      0.66      6701
      benign       0.00      0.00      0.00      6858

    accuracy                           0.49     13559
   macro avg       0.25      0.50      0.33     13559
weighted avg       0.24      0.49      0.33     13559



In [ ]:
### OBFS4 PROXY & SPLITTING

df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic2.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic2.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/combining_2_method/splitting_obfs4.csv', index_col = [0])

df_mal_web_mine.reset_index(drop=True)

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [01:12<00:00,  3.64s/it]


relevance_table  (247, 4)
relevance_table_changed_mal  (255, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.43      1.00      0.61      6701
      benign       0.00      0.00      0.00      8723

    accuracy                           0.43     15424
   macro avg       0.22      0.50      0.30     15424
weighted avg       0.19      0.43      0.26     15424



In [ ]:
### OBFS4 PROXY & PADDING & SPLITTING
df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic2.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic2.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/combining_3_4_method/padding_splitting_obfs4.csv', index_col = [0])

df_mal_web_mine.reset_index(drop=True)

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [01:16<00:00,  3.81s/it]


relevance_table  (247, 4)
relevance_table_changed_mal  (256, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.43      1.00      0.61      6701
      benign       0.00      0.00      0.00      8735

    accuracy                           0.43     15436
   macro avg       0.22      0.50      0.30     15436
weighted avg       0.19      0.43      0.26     15436



In [ ]:
### OBFS4 & DUMMY PACKET & PADDING

df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic2.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic2.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/combining_3_4_method/dummy_packet_padding_obfs4.csv', index_col = [0])

df_mal_web_mine.reset_index(drop=True)

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [01:04<00:00,  3.21s/it]


relevance_table  (247, 4)
relevance_table_changed_mal  (238, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.47      1.00      0.64      6701
      benign       0.00      0.00      0.00      7540

    accuracy                           0.47     14241
   macro avg       0.24      0.50      0.32     14241
weighted avg       0.22      0.47      0.30     14241



In [ ]:
### OBFS4 & DUMMY PACKET & SPLITTING

df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic2.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic2.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/combining_3_4_method/dummy_packet_splitting_obfs4.csv', index_col = [0])

df_mal_web_mine.reset_index(drop=True)

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [01:22<00:00,  4.11s/it]


relevance_table  (247, 4)
relevance_table_changed_mal  (238, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.42      1.00      0.59      6701
      benign       0.00      0.00      0.00      9406

    accuracy                           0.42     16107
   macro avg       0.21      0.50      0.29     16107
weighted avg       0.17      0.42      0.24     16107



In [ ]:
### packet obfs4 proxy & insertion & splitting & replacement 
## combine all
df_malicious = pd.read_csv('./attacked_data/conventional_malicious_traffic2.csv', index_col = [0])
df_benign = pd.read_csv('./attacked_data/benign_traffic2.csv', index_col = [0])
df_mal_web_mine = pd.read_csv('./attacked_data/combining_3_4_method/all_combined.csv', index_col = [0])

df_mal_web_mine.reset_index(drop=True)

print("{} NAN in malicious".format(len(df_malicious[df_malicious.isna().any(axis=1)])))
print("{} NAN in benign".format(len(df_benign[df_benign.isna().any(axis=1)])))

df_malicious = df_malicious.dropna()
df_benign = df_benign.dropna()

start = timer()
results_Raspberry_s1 = run_process(df_malicious,df_benign,df_results, df_mal_web_mine)
end = timer()

0 NAN in malicious
0 NAN in benign


Feature Extraction: 100%|██████████| 20/20 [01:13<00:00,  3.67s/it]


relevance_table  (247, 4)
relevance_table_changed_mal  (243, 4)
let the ml starts
SVM
              precision    recall  f1-score   support

   malignant       0.45      1.00      0.62      6701
      benign       0.00      0.00      0.00      8309

    accuracy                           0.45     15010
   macro avg       0.22      0.50      0.31     15010
weighted avg       0.20      0.45      0.28     15010

